In [1]:
import pandas as pandas
from os import walk
import pyarrow.parquet as parquet

# Used to train document embeddings
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Used to train the baseline model
from sklearn.linear_model import LogisticRegression
import numpy as numpy

# Where the downloaded data are
input_path = '/Users/tyamgin/Projects/mlbootcamp/championship19/data'
# Where to store results
output_path = '/Users/tyamgin/Projects/mlbootcamp/championship19/res/'

In [2]:
# Get the test texts
test_texts = parquet.read_table(input_path + '/texts/textsTest/', \
                                columns = ['objectId','preprocessed']).to_pandas()

/usr/local/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
test_texts.head(10)

,objectId,preprocessed
0,517288,"[квартирник, нтв, маргулис, групп, пилот]"
1,9501964,"[родител, очен, трогательн, песн, артур, халат]"
2,23007371,"[сух, суперджет, откажет, западн, комплект, те..."
3,38353886,"[сгорел, сара, гор, хат]"
4,21192138,"[живодёр, отруб, лап, собак]"
5,26415073,"[ажурн, маков, кулич, так, нежн, воздушн, мяки..."
6,36734526,"[друг, уснул, бар, теб, нужн, тащ, дом, очен, ..."
7,8699823,"[никифор, ден, дат, год, март, понедельник, др..."
8,12236843,[]
9,38393782,"[днр, лнр, новост, войск, берут, кольц, донбас..."


In [4]:
test_texts.preprocessed[2]

array(['сух', 'суперджет', 'откажет', 'западн', 'комплект',
       'технологическ', 'прор', 'росс', 'сух', 'избав', 'западн',
       'начинк', 'модернизац', 'лайнер', 'отечествен', 'комплект',
       'обойдет', 'млрд', 'рубл', 'распоряжен', 'выделен', 'денег',
       'подписа', 'президент', 'росс', 'владимир', 'путин', 'ожида',
       'самолет', 'замен', 'як', 'ту', 'парк', 'специальн', 'летн',
       'отряд', 'росс', 'устаревш', 'тушек', 'парк', 'миноборон', 'машин',
       'намер', 'закуп', 'авиакомпан', 's', 'гражданск', 'перевозок',
       'план', 'уменьш', 'ssj', 'кресел', 'внест', 'изменен', 'крыл',
       'фюзеляж', 'бортов', 'комплекс', 'авионик', 'наш', 'самолет', 'эт',
       'прор', 'самолётостроен', 'бортов', 'оборудован', 'котор', 'сто',
       'самолёт', 'та', 'применен', 'динамик', 'самолёт', 'эт', 'прор',
       'развива', 'сво', 'проект', 'говор', 'президент', 'за',
       'гражданск', 'самолет', 'сух', 'ил', 'тарасенк', 'разрабатыва',
       'проект', 'sukhoi', 'super

In [5]:
# Build document embeddings for text documents
doc2vec = Doc2Vec(\
                [TaggedDocument(lines,'tag') for lines in test_texts.preprocessed], \
                vector_size=5, window=2, min_count=1, workers=4)

In [6]:
test_texts.shape

(667957, 2)

In [7]:
# Read a single day to train model on as Pandas dataframe
data = parquet.read_table(input_path + '/textsTrain/date=2018-02-07', \
                          columns = ['instanceId_objectId','feedback']).to_pandas()

In [8]:
data.head(10)

,instanceId_objectId,feedback
0,22429313,[Ignored]
1,14676953,[Ignored]
2,11562101,[Ignored]
3,20892119,[Clicked]
4,16063005,[Ignored]
5,14576490,[Ignored]
6,11811947,[Ignored]
7,1936012,[Liked]
8,20867189,[Ignored]
9,28967185,[Ignored]


In [9]:
data.rename(columns = {'instanceId_objectId':'objectId'}, inplace = True)

In [10]:
data.head(10)

,objectId,feedback
0,22429313,[Ignored]
1,14676953,[Ignored]
2,11562101,[Ignored]
3,20892119,[Clicked]
4,16063005,[Ignored]
5,14576490,[Ignored]
6,11811947,[Ignored]
7,1936012,[Liked]
8,20867189,[Ignored]
9,28967185,[Ignored]


In [11]:
data['label'] = data['feedback'].apply(lambda x: 1.0 if("Liked" in x) else 0.0).values

In [12]:
data.head(10)

,objectId,feedback,label
0,22429313,[Ignored],0.0
1,14676953,[Ignored],0.0
2,11562101,[Ignored],0.0
3,20892119,[Clicked],0.0
4,16063005,[Ignored],0.0
5,14576490,[Ignored],0.0
6,11811947,[Ignored],0.0
7,1936012,[Liked],1.0
8,20867189,[Ignored],0.0
9,28967185,[Ignored],0.0


In [13]:
data = data[['objectId','label']]
data.head(10)

,objectId,label
0,22429313,0.0
1,14676953,0.0
2,11562101,0.0
3,20892119,0.0
4,16063005,0.0
5,14576490,0.0
6,11811947,0.0
7,1936012,1.0
8,20867189,0.0
9,28967185,0.0


In [14]:
parts = []

# Get unique object ids
ids = data.groupby('objectId').count()

# In order to save memory iterate part by part
for (dirpath, dirnames, filenames) in walk(input_path + '/texts/textsTrain/'):
    for name in filenames:
        if name.startswith('part'):
            # Read single part
            texts = parquet.read_table(input_path + '/texts/textsTrain/' + name, \
                                       columns = ['objectId','preprocessed']).to_pandas()            
            # Filter documents we need
            joined = ids.join(texts.set_index('objectId'), how='inner', on = 'objectId')
            # Evaluate embeddings
            joined['embedding'] = joined.preprocessed.apply(doc2vec.infer_vector)
            # Memorize
            parts.append(joined[['embedding']])
            print('Done with ' + name)

Done with part-00026-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00011-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00002-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00013-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00024-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00000-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00006-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00022-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00015-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00004-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00017-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00020-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00008-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00019-1b50c8f5-87db-4a5

In [20]:
# Combine all the parts
train = data.join(pandas.concat(parts), on = 'objectId')
train.head(10)

,objectId,label,embedding
0,22429313,0.0,"[-0.44170317, 1.7940384, 2.5307617, 0.21553585..."
1,14676953,0.0,"[-0.10807835, 1.1627201, 1.7504562, 0.24641608..."
2,11562101,0.0,"[0.013378491, -0.26066193, 0.78019893, 0.11125..."
3,20892119,0.0,"[-0.05681707, 0.17438778, 1.2145396, -0.040171..."
4,16063005,0.0,"[-0.35590836, 1.6147468, 3.2192364, 0.06583231..."
5,14576490,0.0,"[-0.12549248, 0.097653374, 0.08917581, 0.00392..."
6,11811947,0.0,"[0.16296238, 0.024091508, 0.9208572, 0.0075760..."
7,1936012,1.0,"[0.07246128, -0.07718521, 0.565532, -0.1438746..."
8,20867189,0.0,"[0.010391599, 1.4388258, 1.4911665, -0.2362101..."
9,28967185,0.0,"[0.09291192, 0.24640752, 0.003460342, 0.017822..."


In [21]:
# Construct the label (liked objects)
y = train['label'].values

In [22]:
y

array([0., 0., 0., ..., 1., 0., 1.])

In [40]:
# Extract the most interesting features
X = numpy.stack(train['embedding'].values)

In [41]:
X

array([[-0.44170317,  1.7940384 ,  2.5307617 ,  0.21553585, -1.8123595 ],
       [-0.10807835,  1.1627201 ,  1.7504562 ,  0.24641608, -1.5539731 ],
       [ 0.01337849, -0.26066193,  0.78019893,  0.11125397,  0.10340756],
       ...,
       [ 0.11090618,  0.16961716,  0.25980642, -0.03150712,  0.00698385],
       [-0.05458339, -0.09454934,  0.26065007, -0.22154804, -0.0865216 ],
       [ 0.02650395,  0.02290959,  0.7163141 , -0.19513054, -0.13866283]],
      dtype=float32)

In [42]:
# Fit the model and check the weights
model = LogisticRegression(random_state=0, solver='lbfgs').fit(X, y)
model.coef_

array([[-0.07185373, -0.04516869, -0.00781221, -0.67337679,  0.19353822]])

In [45]:
proba_result = model.predict_proba(numpy.stack(\
    test_texts.preprocessed.apply(lambda x : doc2vec.infer_vector(x))))

In [50]:
test_texts['weight'] = -proba_result[:, 1]

In [51]:
test_texts.head(10)

,objectId,preprocessed,weight
0,517288,"[квартирник, нтв, маргулис, групп, пилот]",-0.167617
1,9501964,"[родител, очен, трогательн, песн, артур, халат]",-0.184983
2,23007371,"[сух, суперджет, откажет, западн, комплект, те...",-0.177372
3,38353886,"[сгорел, сара, гор, хат]",-0.175593
4,21192138,"[живодёр, отруб, лап, собак]",-0.189251
5,26415073,"[ажурн, маков, кулич, так, нежн, воздушн, мяки...",-0.090058
6,36734526,"[друг, уснул, бар, теб, нужн, тащ, дом, очен, ...",-0.163667
7,8699823,"[никифор, ден, дат, год, март, понедельник, др...",-0.246918
8,12236843,[],-0.177239
9,38393782,"[днр, лнр, новост, войск, берут, кольц, донбас...",-0.201176


In [52]:
# Read the test data
test = parquet.read_table(input_path + '/textsTest', \
    columns = ['instanceId_userId','instanceId_objectId']).to_pandas()
test.rename(columns = {'instanceId_objectId':'objectId'}, inplace = True)
test.head(10)

,instanceId_userId,objectId
0,1006,34577503
1,1006,37520199
2,1618,546086
3,1618,546086
4,1618,546086
5,1618,35981492
6,1618,26764305
7,1810,958605
8,1810,20479574
9,1810,36254478


In [108]:
test_texts.query('objectId==10672856')

,objectId,preprocessed,weight
454278,10672856,"[помога, инач, поможет, выслуша, её, инач, её,...",-0.188005
497246,10672856,"[помога, инач, поможет, выслуша, её, инач, её,...",-0.160773


In [106]:
test_texts.query('objectId==10672856').iloc[0].preprocessed

array(['помога', 'инач', 'поможет', 'выслуша', 'её', 'инач', 'её',
       'выслуша', 'бал', 'дар', 'цвет', 'обнима', 'прост', 'буд', 'ряд',
       'ряд', 'найдет', 'вмест', 'стира', 'слёзы', 'лиц', 'стира', 'жизн',
       'люд', 'котор', 'застав', 'плака', 'отношен', 'будущ', 'продл',
       'ровн', 'стольк', 'скольк', 'женщин', 'хват', 'терпен', 'ничт',
       'обижа', 'равнодуш', 'мужчин', 'женщин', 'плачет', 'мужчин',
       'долж', 'понима', 'слез', 'эт', 'уловк', 'катаклизм', 'миров',
       'катастроф', 'взорва', 'вулка', 'рек', 'вышл', 'берег', 'немедлен',
       'люб', 'способ', 'останов', 'плачет', 'говор', 'стира', 'слез',
       'прост', 'обн', 'поцел', 'рук', 'держ', 'настольк', 'нежн',
       'наскольк', 'сможеш', 'тех', 'пор', 'пок', 'тво', 'любов',
       'дойдет', 'сердц', 'да', 'сил', 'душ', 'позвол', 'почувствова',
       'кажд', 'миг', 'счаст', 'гор', 'жизн', 'смерт', 'земл', 'неб'],
      dtype=object)

In [107]:
test_texts.query('objectId==10672856').iloc[1].preprocessed

array(['помога', 'инач', 'поможет', 'выслуша', 'её', 'инач', 'её',
       'выслуша', 'бал', 'дар', 'цвет', 'обнима', 'прост', 'буд', 'ряд',
       'ряд', 'найдет', 'вмест', 'стира', 'слёзы', 'лиц', 'стира', 'жизн',
       'люд', 'котор', 'застав', 'плака', 'отношен', 'будущ', 'продл',
       'ровн', 'стольк', 'скольк', 'женщин', 'хват', 'терпен', 'ничт',
       'обижа', 'равнодуш', 'мужчин', 'женщин', 'плачет', 'мужчин',
       'долж', 'понима', 'слез', 'эт', 'уловк', 'катаклизм', 'миров',
       'катастроф', 'взорва', 'вулка', 'рек', 'вышл', 'берег', 'немедлен',
       'люб', 'способ', 'останов', 'плачет', 'говор', 'стира', 'слез',
       'прост', 'обн', 'поцел', 'рук', 'держ', 'настольк', 'нежн',
       'наскольк', 'сможеш', 'тех', 'пор', 'пок', 'тво', 'любов',
       'дойдет', 'сердц', 'да', 'сил', 'душ', 'позвол', 'почувствова',
       'кажд', 'миг', 'счаст', 'гор', 'жизн', 'смерт', 'земл', 'неб',
       'всё', 'связа'], dtype=object)

In [93]:
test.join(\
        test_texts[['objectId','weight']].set_index('objectId'), \
        how = 'inner', \
        on = 'objectId') \
    .query('objectId==10672856 & instanceId_userId==742')

,instanceId_userId,objectId,weight
306033,742,10672856,-0.188005
306033,742,10672856,-0.160773
306034,742,10672856,-0.188005
306034,742,10672856,-0.160773


In [112]:
# Join test documents and elliminate possible duplicates
scores = test.join(\
    test_texts[['objectId','weight']].set_index('objectId'), \
    how = 'inner', \
    on = 'objectId') \
    .groupby(['instanceId_userId','objectId']).min()
scores.head(10)

weight
instanceId_userId objectId          
316               17997084 -0.145631
                  37758420 -0.146873
631               15478935 -0.160086
                  30513650 -0.185748
                  38118098 -0.158939
742               10672856 -0.188005
                  24302446 -0.162636
                  28816291 -0.177682
                  34685448 -0.148833
868               11640701 -0.172537

In [114]:
#  Sort for each user
result = scores.sort_values(by=['instanceId_userId', 'weight']).reset_index()
result.head(10)

,instanceId_userId,objectId,weight
0,316,37758420,-0.146873
1,316,17997084,-0.145631
2,631,30513650,-0.185748
3,631,15478935,-0.160086
4,631,38118098,-0.158939
5,742,10672856,-0.188005
6,742,28816291,-0.177682
7,742,24302446,-0.162636
8,742,34685448,-0.148833
9,868,30882080,-0.190935


In [118]:
# Collect predictions for each user
submit = result.groupby("instanceId_userId")['objectId'].apply(list)
submit.head(10)

instanceId_userId
316                                  [37758420, 17997084]
631                        [30513650, 15478935, 38118098]
742              [10672856, 28816291, 24302446, 34685448]
868     [30882080, 30143153, 22115500, 11640701, 29193...
979                                   [37950972, 7996257]
1006                                 [34577503, 37520199]
1276                       [36856262, 31000576, 22812401]
1444                                 [36806487, 20963755]
1483                                 [38036543, 34991228]
1618                         [546086, 35981492, 26764305]
Name: objectId, dtype: object

In [119]:
result.shape

(1035047, 3)

In [121]:
# Persist the first submit
submit.to_csv(output_path + "/textSubmit.csv.gz", header = False, compression='gzip')

In [123]:
joined.preprocessed[1:5]

objectId
300695    [торт, минут, выпечк, вкус, детств, торт, крек...
302521                   [чуж, премьер, песн, брав, яросла]
302773    [ландорик, блюд, мо, детств, подписыва, видеок...
304557                                     [зажига, мужчин]
Name: preprocessed, dtype: object